In [18]:
#IMPORT NECESSARY LIBRARIES
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
import spacy

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import IsolationForest

In [3]:
df = pd.read_csv("Datasets/amazon_reviews2.csv")

In [4]:
df.columns

Index(['Unnamed: 0', 'url', 'review_title', 'ratings', 'review', 'verified',
       'date_x', 'user_name', 'user_id', 'max_reviews_day', 'by_link',
       'helpful_votes', 'product', 'product_link', 'review_sentiment',
       'total_reviews', 'average_rating', 'rating_deviation', 'review_length',
       'title_length', 'total_user_reviews', 'datetime', 'review_date_diff'],
      dtype='object')

In [6]:
len(df)

7652

In [9]:
#PREPROCESS THE TEXT OF REVIEWS AND TITLES
nlp = spacy.load('en_core_web_sm')
stemmer = PorterStemmer()
stop_words = nlp.Defaults.stop_words

def preprocess_text(text):
    doc = nlp(text)
    preprocessed_tokens = [
        token.lemma_ for token in doc if token.text.lower() not in stop_words
        ]
    return " ".join([stemmer.stem(token) for token in preprocessed_tokens])



In [10]:
#PREPROCESS TEXT OF REVIEW
df['review'] = df['review'].apply(preprocess_text)

In [12]:
df['review_title'] = df['review_title'].apply(preprocess_text)

In [23]:
X = df[[
    'ratings','verified',
    'max_reviews_day',
    'helpful_votes', 'review_sentiment',
    'total_reviews', 'average_rating', 'rating_deviation', 'review_length',
    'title_length', 'total_user_reviews', 'review_date_diff'
]]

In [26]:
X.isnull().sum()

ratings                 0
verified                0
max_reviews_day         0
helpful_votes           0
review_sentiment        0
total_reviews         317
average_rating        317
rating_deviation      317
review_length           0
title_length            0
total_user_reviews      0
review_date_diff        0
dtype: int64

In [28]:
X.fillna(0, inplace=True)

/var/folders/tl/ps_r0r591fx137cf06vhn3tm0000gn/T/ipykernel_10705/2534066105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.fillna(0, inplace=True)


In [17]:
#ISOLATION FOREST FOR ANOMALY DETECTION
model = IsolationForest(random_state = 42)

In [29]:
#GRID SEARCH TO FIND OPTIMAL N_ESTIMATORS

#POSSIBLE VALUES
param_grid = {'n_estimators': [25, 30, 40, 50, 70, 100,  150, 175, 200]}

#GRID CREATION
grid_search = GridSearchCV(model, param_grid, cv = 5, scoring='accuracy')

#FIT THE GRID ON FEATURES
grid_search.fit(X)

best_n_estimators = grid_search.best_params_['n_estimators']

/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argumen

/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argumen

/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
TypeError: _BaseScorer.__call__() missing 1 required positional argumen

In [30]:
best_n_estimators

25

In [31]:
model = IsolationForest(n_estimators = 25, random_state = 42)

In [32]:
#FIT MODEL ON FEATURE MATRIX
model.fit(X)

#PREDICT ANOMALY SCORES FOR EACH X
anomaly_scores = model.decision_function(X)

#BINARY LABELS [1 FOR NORMAL; -1 FOR ANOMALY]
labels = model.predict(X)

In [35]:
labels
df['IF_LABEL'] = labels

In [36]:
df['IF_LABEL'].value_counts()

IF_LABEL
 1    6671
-1     981
Name: count, dtype: int64